# Vanishing Gradient problem and Weight Initialization 

Well. You have implemented a simple logistic regression model and a two layer fully connected shallow network for cat vs non-cat classification. You would have found that the shallow network does not significantly outperform logistic regression. Further, you might conclude based on the shallow net's performance of close to 100% training accuracy and around 72% test accuracy that it is heavily overfitting. So you would have tried L1/L2 regularization or dropout regularization. To your surprise, you would have found that regularization (L1, L2 or dropout) did not have any effect or in fact exhibited a negative effect by pulling down model's generalization ability.

Is it a surprise?? What was the performance of logistic regression? Is it not close to 100% training accuracy and around 70% test accuracy? So, is logistic regression overfitting? In fact it is the simplest model with smallest number of parameters. Zeroing out more parameters will increase the bias of the model thereby resulting in much lower training accuracy as well as test accuracy. Test this case for yourself by randomly setting few parameters in logistic regression model to zero prior to training or, even during training like dropout. Make sure the activations are scaled as in dropout if you are trying the latter case.

This understanding suggests that model requires more capacity to generalize better. That is, we need to try a deeper network to improve generalization capability. Subsequently we can enforce regularization to constrain weights with smaller norm.

## Exercise
i. Build a model based on the design in assgn_2b for the following architecture:
        
        I/p---->Linear1---->Sigmoid----->Linear2---->Sigmoid---->Linear3---->Sigmoid---->Linear4---->Sigmoid
   where: 
       
       Linear1 has 64*64*3=12288 incoming connections and 30 outgoing connections
       Linear2 has 30 incoming connections and 30 outgoing connections
       Linear3 has 30 incoming  connections and 30 outgoing connections 
       Linear4 has 30 incoming  connections and 1 outgoing connection 

In [20]:
import numpy as np
import argparse # for command-line parsing
import matplotlib # for plotting
from matplotlib import pyplot as plt # for plotting
from abc import ABC, abstractmethod # for crating abstract classes

from assign2_utils import load_train_data, load_test_data, flatten, df 

%matplotlib inline 

In [21]:
class Layers(ABC): # Layers inherits from ABC.The class ABC in abc module is required to make a class abstract. 
                   # Native python does not support abstract classes
    def __init__(self): 
        super().__init__() # calls the constructor of the super class ABC
    
    @abstractmethod
    def forward(self, *x): # should be overridden by every class that inherits from this class
        pass
    
    @abstractmethod
    def backward(self, *da): # should be overridden by every class that inherits from this class
        pass    
    
    def update_params(self, learning_rate = None): # should be overridden by those inheriting classes that have
                                                   # parameters to be learnt
        raise NotImplementedError
    
    def __call__(self, *x): # makes the inheriting classes callable, calling the forward method of the class
        return self.forward(*x)

In [22]:
class Linear(Layers): # inherits from abstract class Layers
    def __init__(self, in_features, out_features, bias = True, regularization = None): # constructor
        super().__init__()
        self.in_features = in_features # initialize number of incoming features
        self.out_features = out_features # initialize number of outgoing features
        self.weight = np.random.randn(out_features, in_features) * .01 # initialize weight matrix
        if bias:
            self.bias = np.zeros((out_features, 1)) # initialize bias if required
        else:
            self.bias = None # if bias not required, set it to None
        self.regularization = regularization # initialize regularization
        if self.regularization is not None: 
            self.reg_penalty = 0 # if valid regularization at this layer, set the regularization penalty
                                 # at this layer initially to zero
            
        self.dw = np.empty_like(self.weight) # intialize dw to empty
        self.db = np.empty_like(self.bias) if self.bias is not None else None 
                                            # initialize db to empty if bias == True
        
    def forward(self, x): # forward method overridden; x is the incoming activation. 
                          # shape of x is (num of activations, num of samples)
            
#         print("Linear Forward")
        m = x.shape[1]    # number of training examples
        self.x = x # x is required for backward. We don't need a separate cache. We can store it in the object.
        
        output = self.weight @ x # computation of the linear part Wx                                 
        if self.bias is not None:            
            output += self.bias # add to Wx bias if bias == True
                                # Note that we don't apply non-linearity here as this layer computes only Wx+b
        
        #update regularization penalty at this layer
        if self.regularization is None:
            pass
        elif self.regularization == 'L2':
            self.reg_penalty = args.lamda/(2*m) * np.sum(self.weight**2)
        elif self.regularization == 'L1':
            self.reg_penalty = args.lamda/m * np.sum(np.abs(self.weight))
        else:
            raise ValueError(f'Regularization method{self.regularization} not defined')
            
        return output # return forward output as the next layer in the model will require it     
    
    # Backward of this layer will receive dz. Note that at this layer z = Wx+b. So backward will compute
    # dw, db and dx. To compute dW, x is required. That's why x was stored in forward. To compute dx, W is 
    # required. This is already available in self.weight
    
    def backward(self, dz): # backward method overridden
        m = dz.shape[1]     # number of training examples
#         print("Linear Backward")
        self.dw = dz @ self.x.T # compute dw
        
        # add derivative of regularization penalty at this layer w.r.to w
        if self.regularization is None:
            pass
        elif self.regularization == 'L1':
            signw = np.sign(self.weight)
            signw[signw == 0] = 1
            self.dw += args.lamda/m * signw
        elif self.regularization == 'L2':
            self.dw += args.lamda/m * self.weight
        else:
            raise ValueError(f'Regularization method{self.regularization} not defined')
            
        if self.bias is not None: # compute db if bias == True
            self.db = np.sum(dz, axis = 1, keepdims = True) 
            
        dx = self.weight.T @ dz  # compute dx     
        return dx # we only return dx as this this required for chain rule in the next layer 
                  # in backward direction. dw and db are kept available in this object which will directly
                  # be used by update_params for updating weights and biases
    
    # update parameters in this layer 
    def update_params(self, learning_rate = 0.005):
        self.weight -= learning_rate*self.dw
        if self.bias is not None:
            self.bias -= learning_rate*self.db

In [23]:
class NonLinear(Layers):
    def __init__(self, fname='ReLU'):
        super().__init__()
        self.fname = fname
    
    def forward(self, x):
#         print("NonLinear Forward")
        self.x = x
        if self.fname == 'ReLU':
            return np.maximum(x, 0)
        elif self.fname == 'Sigmoid':
            return 1 / (1 + np.exp(-x))
        elif self.fname == 'Tanh':
            return np.tanh(x)
        else:
            raise ValueError('Unknown non-linear function error')
    
    def backward(self, dx):    # implemented instead of using df(...)
#         print("NonLinear Backward")
        if self.fname == 'ReLU':
            return dx * (self.x > 0)
        elif self.fname == 'Sigmoid':
            sigmoid_x = 1 / (1 + np.exp(-self.x))
            return dx * sigmoid_x * (1 - sigmoid_x)
        elif self.fname == 'Tanh':
            return dx * (1 - np.tanh(self.x)**2)
        else:
            raise ValueError('Unknown non-linear function error')
    
    def update_params(self, learning_rate=None):
        pass
    

In [24]:
class Dropout(Layers):
    def __init__(self, keep_prob = 0.8):
        super().__init__()
        self.keep_prob = keep_prob
    
    def forward(self, x, train=True):
        if train:    
            d = np.random.rand(*x.shape)
            d = (d < self.keep_prob)
            self.d = d
            x = x * d
            x = x / self.keep_prob
        return x
    
    def backward(self, dx):
        return dx * self.d
    
    def __call__(self, *x, train=True):
        return self.forward(*x, train=train)

In [25]:
class Model(Layers):
    def __init__(self, in_features):
        super().__init__()
        # self.fc1 = Linear(in_features, 32, regularization = 'L2')
        self.fc1 = Linear(in_features, 30)
        self.sigmoid1 = NonLinear('Sigmoid')
        self.fc2 = Linear(30, 30)
        self.sigmoid2 = NonLinear('Sigmoid')
        self.fc3 = Linear(30, 30)
        self.sigmoid3 = NonLinear('Sigmoid')
        self.fc4 = Linear(30, 1)
        self.sigmoid4 = NonLinear('Sigmoid')
        
    def forward(self, x, train=True):
        x = self.fc1(x) # Note that we made classes callable which automatically calls forward method
                        # That's why we could call fc1(x) instead of fc1.forward(x). Calls below 
                        # are in similar line.
                        # we could call fc1.forward(x) also.
        x = self.sigmoid1(x)
        x = self.fc2(x)
        x = self.sigmoid2(x)
        x = self.fc3(x)
        x = self.sigmoid3(x)
        x = self.fc4(x)
        x = self.sigmoid4(x)
        return x
    
    def loss(self, output, y):
        m = output.shape[1]
        L = -(1./m) * np.sum(y*np.log(output) + (1-y)*np.log(1-output)) # compute loss
        for att in self.__dict__:
            if hasattr(att, 'reg_penalty'):
                L += att.reg_penalty
        return L
    
    def backward(self, output, y):
        epsilon = 1e-6
        m = output.shape[1]
        d_output = (1./m) * (output-y) * (1./((output*(1-output))+epsilon)) # compute da        
        dz = self.sigmoid4.backward(d_output)
        dx = self.fc4.backward(dz)
        dz = self.sigmoid3.backward(dx)
        dx = self.fc3.backward(dz)
        dz = self.sigmoid2.backward(dx)
        dx = self.fc2.backward(dz)
        dz = self.sigmoid1.backward(dx)
        dx = self.fc1.backward(dz)
    
    def update_params(self, learning_rate = 0.005):
        self.fc1.update_params(learning_rate)
        self.fc2.update_params(learning_rate)
        self.fc3.update_params(learning_rate)
        self.fc4.update_params(learning_rate)
        
    
    def __call__(self, *x, train=True):
        return self.forward(*x, train=train)

In [7]:
# instantiate the ArgumentParser object
parser = argparse.ArgumentParser(description='Train a fully connected network with regularization')
# add arguments
parser.add_argument('--miter', metavar='N', type=int, default=200, help='max number of iterations to train')
parser.add_argument('--alpha', metavar='LEARNING_RATE', type=float, default=0.001, help='initial learning rate')
parser.add_argument('--lamda', metavar='LAMBDA', type=float, default=1., help='regularization parameter')
parser.add_argument('--print_freq', metavar='N', type=int, default=300, help='print model loss every print_freq iterations')

# parse the arguments. 
# Since we cannot invoke the code written in jupyter directly from command-line, 
# we can specify the required arguments in the call to parse_args as shown below with other arguments 
# left out to use their default values.
args = parser.parse_args('--miter 2000 --alpha .008'.split()) # you may play with this code by changing
                                                                        # the arguments as required

In [26]:
def train(model, x, y):
    for i in range(args.miter):
        output = model(x) # model is a callable object with call to its forward method.
                          # we could also have written the rhs as model.forward(x)
        L = model.loss(output, y)
        model.backward(output, y)
        model.update_params(args.alpha)
        if not i%args.print_freq: # print loss every 100 iterations
                print(f'Loss at iteration {i}:\t{np.asscalar(L):.4f}')
                
def test_model(model, x, y):
    predictions = model(x, train=False)
    predictions[predictions > 0.5] = 1
    predictions[predictions <= 0.5] = 0
    acc = np.mean(predictions == y)
    acc = np.asscalar(acc)
    return acc

In [9]:
def main(): # main function to train and test the model    
    
    global args
    # load train data
    x, y = load_train_data()
    x = flatten(x)
    x = x/255. # normalize the data to [0, 1]     
    
    # Instantiate the model
    my_model = Model(x.shape[0])
    
    # train the model
    train(my_model, x, y)
    
    # test the model
    print(f'train accuracy: {test_model(my_model, x, y) * 100:.2f}%')

    x, y = load_test_data()
    x = flatten(x)
    x = x/255. # normalize the data to [0, 1]
    print(f'test accuracy: {test_model(my_model, x, y) * 100:.2f}%')
    
    return
    
if __name__ == '__main__':
    main()

Loss at iteration 0:	0.6953
Loss at iteration 300:	0.6440
Loss at iteration 600:	0.6440
Loss at iteration 900:	0.6440
Loss at iteration 1200:	0.6440
Loss at iteration 1500:	0.6440
Loss at iteration 1800:	0.6440
train accuracy: 65.55%
test accuracy: 34.00%


        
ii. Build a model based on the design in assgn_2b for the following architecture:
        
        I/p---->Linear1---->Sigmoid----->Linear2---->Sigmoid---->Linear3----Sigmoid---->Linear4---->Sigmoid--->                                                                                         Linear5---->Sigmoid      
    where: 
    
       Linear1 has 64*64*3=12288 incoming connections and 30 outgoing connections
       Linear2 has 30 incoming connections and 30 outgoing connections
       Linear3 has 30 incoming  connections and 30 outgoing connections 
       Linear4 has 30 incoming  connections and 30 outgoing connections
       Linear5 has 30 incoming  connections and 1 outgoing connection

You can set learning_rate to 0.008 and run for around 2000 iterations.
What is the training and test accuracies in both the cases? Are these deeper models performing better?

In [10]:
class Model(Layers):
    def __init__(self, in_features):
        super().__init__()
        # self.fc1 = Linear(in_features, 32, regularization = 'L2')
        self.fc1 = Linear(in_features, 30)
        self.sigmoid1 = NonLinear('Sigmoid')
        self.fc2 = Linear(30, 30)
        self.sigmoid2 = NonLinear('Sigmoid')
        self.fc3 = Linear(30, 30)
        self.sigmoid3 = NonLinear('Sigmoid')
        self.fc4 = Linear(30, 30)
        self.sigmoid4 = NonLinear('Sigmoid')
        self.fc5 = Linear(30, 1)
        self.sigmoid5 = NonLinear('Sigmoid')
        
    def forward(self, x, train=True):
        x = self.fc1(x) # Note that we made classes callable which automatically calls forward method
                        # That's why we could call fc1(x) instead of fc1.forward(x). Calls below 
                        # are in similar line.
                        # we could call fc1.forward(x) also.
        x = self.sigmoid1(x)
        x = self.fc2(x)
        x = self.sigmoid2(x)
        x = self.fc3(x)
        x = self.sigmoid3(x)
        x = self.fc4(x)
        x = self.sigmoid4(x)
        x = self.fc5(x)
        x = self.sigmoid5(x)
        return x
    
    def loss(self, output, y):
        m = output.shape[1]
        L = -(1./m) * np.sum(y*np.log(output) + (1-y)*np.log(1-output)) # compute loss
        for att in self.__dict__:
            if hasattr(att, 'reg_penalty'):
                L += att.reg_penalty
        return L
    
    def backward(self, output, y):
        epsilon = 1e-6
        m = output.shape[1]
        d_output = (1./m) * (output-y) * (1./((output*(1-output))+epsilon)) # compute da        
        dz = self.sigmoid5.backward(d_output)
        dx = self.fc5.backward(dz)
        dz = self.sigmoid4.backward(dx)
        dx = self.fc4.backward(dz)
        dz = self.sigmoid3.backward(dx)
        dx = self.fc3.backward(dz)
        dz = self.sigmoid2.backward(dx)
        dx = self.fc2.backward(dz)
        dz = self.sigmoid1.backward(dx)
        dx = self.fc1.backward(dz)
    
    def update_params(self, learning_rate = 0.005):
        self.fc1.update_params(learning_rate)
        self.fc2.update_params(learning_rate)
        self.fc3.update_params(learning_rate)
        self.fc4.update_params(learning_rate)
        self.fc5.update_params(learning_rate)
        
    
    def __call__(self, *x, train=True):
        return self.forward(*x, train=train)

In [11]:
# instantiate the ArgumentParser object
parser = argparse.ArgumentParser(description='Train a fully connected network with regularization')
# add arguments
parser.add_argument('--miter', metavar='N', type=int, default=200, help='max number of iterations to train')
parser.add_argument('--alpha', metavar='LEARNING_RATE', type=float, default=0.001, help='initial learning rate')
parser.add_argument('--lamda', metavar='LAMBDA', type=float, default=1., help='regularization parameter')
parser.add_argument('--print_freq', metavar='N', type=int, default=300, help='print model loss every print_freq iterations')

# parse the arguments. 
# Since we cannot invoke the code written in jupyter directly from command-line, 
# we can specify the required arguments in the call to parse_args as shown below with other arguments 
# left out to use their default values.
args = parser.parse_args('--miter 2000 --alpha .008'.split()) # you may play with this code by changing
                                                                        # the arguments as required

In [12]:
def main(): # main function to train and test the model    
    
    global args
    # load train data
    x, y = load_train_data()
    x = flatten(x)
    x = x/255. # normalize the data to [0, 1]     
    
    # Instantiate the model
    my_model = Model(x.shape[0])
    
    # train the model
    train(my_model, x, y)
    
    # test the model
    print(f'train accuracy: {test_model(my_model, x, y) * 100:.2f}%')

    x, y = load_test_data()
    x = flatten(x)
    x = x/255. # normalize the data to [0, 1]
    print(f'test accuracy: {test_model(my_model, x, y) * 100:.2f}%')
    
    return
    
if __name__ == '__main__':
    main()

Loss at iteration 0:	0.6939
Loss at iteration 300:	0.6440
Loss at iteration 600:	0.6440
Loss at iteration 900:	0.6440
Loss at iteration 1200:	0.6440
Loss at iteration 1500:	0.6440
Loss at iteration 1800:	0.6440
train accuracy: 65.55%
test accuracy: 34.00%


## Vanishing Gradient Problem
To understand this, let's look at the plot shown below. This plot displays the size of dw, ie $||dw||_{2}$ at every neuron at hidden layers 1, 2 and 3 of the model that is solution to the first exercise above. 
<img src='dw_sigmoid_4layers.png'>

You find that, compared to layer3, size of $dw$ at the earlier layers (layers 1 and 2), particularly layer 1, have mostly very neglible value implying that $w$ at earlier layers update minimally compared to later layers. In fact, across training, you find from the above plot that most of the neurons from layer 1 update very minimally and so learn very slowly. Also, the magnitude of size of dw has come down by order of $10^3$ across training. Ok, hold this in your mind and let's look at the plots for solution of exercise 2 given above. This model is deeper than the model corresponding to exercise 1. 
<img src='dw_sigmoid_5layers.png'>


You see a similar behaviour but with more severity. The magnitudes have come down by an order of $10^5$. You must have observed that, when you trained your solutions to both exercises given above, training almost loss plateaus after few iterations and does not significantly decrease over iterations. To summarise, we find that earlier layers in a deeper network learns very slow and the rate of slowness in learning increases with depth. The figure below depicts the speed of learning w.r.to number of epochs across all hidden layers in the solution to exercise 2. We can see that earlier layers 1 and 2 do not learn right from the beginning. Later layers also stop updating significantly after around 400 epochs. Is this behaviour random or expected??? It cannot be random since this happened for your solutions and all your classmates solutions as well!! So why did this happen? 
<img src='speed_learning.png'>

The gradient of loss w.r.to $w$, $dW$, decreases in magnitude significantly for earlier layers i.e the gradient is vanishing. You might wonder vanishing gradient is favourable since it seems to mean that optimum weights have been arrived at (from calculus, when derivative of f is zero at x , then x is a stationary point). But remember that weights have been initialized randomly and so it is highly unlikely that these weights will be the optimimum ones. These weights at earlier layers hardly get updated even though at later layers significant updates happen. Random initialization mean that input features are randomly combined at layer 1 leading to a significant likelihood of loss of input information. So, even if later layers learn, they will still underperform in recognizing input as they do not have enough information about input right from the beginning. so, why is the gradient vanishing fast for earlier layers?

To understand this let's consider the $dW$ equation at layer $l$ in backpropagation:
<br><br>
&emsp;$dW^{[l]}$ = $dz^{[l]}$.${a^{[l-1]}}^T$ where . is matrix multiplication and T denotes transpose
<br><br>
Here, 
<br><br>
&emsp;$da^{[l]}$ = ${dW^{[l+1]}}^T$.$dz^{[l+1]}$ for $1<=l<L$ and $da^{[L]}$ = $\frac{1.0}{m}(a^{[L]}-y)*\frac{1.0}{a^{[L]}*(1-a^{[L]})}$
<br><br>
Specifically, for the model in exercise 2, we have, 
<br><br>
$dW^{[5]}$ = $dz^{[5]}$.${a^{[4]}}^T$
<br>
&emsp;&nbsp; &nbsp; &nbsp; = $da^{[5]}*\color{blue}{\sigma ^{'}(z^{[5]})}.{a^{[4]}}^T$
<br><br>
$dW^{[4]}$ = $dz^{[4]}$.${a^{[3]}}^T$
<br>
&emsp;&nbsp; &nbsp; &nbsp; = $da^{[4]}*\sigma ^{'}(z^{[4]}).{a^{[3]}}^T$
<br>
&emsp;&nbsp; &nbsp; &nbsp; = ${dW^{[5]}}^T.dz^{[5]}*\sigma ^{'}(z^{[4]}).{a^{[3]}}^T$
<br>
&emsp;&nbsp; &nbsp; &nbsp; = $({da^{[5]}*\color{blue}{\sigma ^{'}(z^{[5]})}.{a^{[4]}}^T})^T.dz^{[5]}*\color{blue}{\sigma ^{'}(z^{[4]})}.{a^{[3]}}^T$
<br><br>
$dW^{[3]}$ = $dz^{[3]}$.${a^{[2]}}^T$
<br>
&emsp;&nbsp; &nbsp; &nbsp; = $da^{[3]}*\sigma ^{'}(z^{[3]}).{a^{[2]}}^T$
<br>
&emsp;&nbsp; &nbsp; &nbsp; = ${dW^{[4]}}^T.dz^{[4]}*\sigma ^{'}(z^{[3]}).{a^{[2]}}^T$
<br>
&emsp;&nbsp; &nbsp; &nbsp; = $({({da^{[5]}*\color{blue}{\sigma ^{'}(z^{[5]})}.{a^{[4]}}^T})^T.dz^{[5]}*\color{blue}{\sigma ^{'}(z^{[4]})}.{a^{[3]}}^T})^T.dz^{[4]}*\color{blue}{\sigma ^{'}(z^{[3]})}.{a^{[2]}}^T$
<br><br>
$dW^{[2]}$ = $dz^{[2]}$.${a^{[1]}}^T$
<br>
&emsp;&nbsp; &nbsp; &nbsp; = $da^{[2]}*\sigma ^{'}(z^{[2]}).{a^{[1]}}^T$
<br>
&emsp;&nbsp; &nbsp; &nbsp; = ${dW^{[3]}}^T.dz^{[3]}*\sigma ^{'}(z^{[2]}).{a^{[1]}}^T$
<br>
&emsp;&nbsp; &nbsp; &nbsp; = $({({({da^{[5]}*\color{blue}{\sigma ^{'}(z^{[5]})}.{a^{[4]}}^T})^T.dz^{[5]}*\color{blue}{\sigma ^{'}(z^{[4]})}.{a^{[3]}}^T})^T.dz^{[4]}*\color{blue}{\sigma ^{'}(z^{[3]})}.{a^{[2]}}^T})^T.dz^{[3]}*\color{blue}{\sigma ^{'}(z^{[2]})}.{a^{[1]}}^T$
<br><br>
$dW^{[1]}$ = $dz^{[1]}$.${a^{[0]}}^T$
<br>
&emsp;&nbsp; &nbsp; &nbsp; = $da^{[1]}*\sigma ^{'}(z^{[1]}).{a^{[0]}}^T$
<br>
&emsp;&nbsp; &nbsp; &nbsp; = ${dW^{[2]}}^T.dz^{[2]}*\sigma ^{'}(z^{[2]}).{a^{[0]}}^T$
<br>
&emsp;&nbsp; &nbsp; &nbsp; = $({({({({da^{[5]}*\color{blue}{\sigma ^{'}(z^{[5]})}.{a^{[4]}}^T})^T.dz^{[5]}*\color{blue}{\sigma ^{'}(z^{[4]})}.{a^{[3]}}^T})^T.dz^{[4]}*\color{blue}{\sigma ^{'}(z^{[3]})}.{a^{[2]}}^T})^T.dz^{[3]}*\color{blue}{\sigma ^{'}(z^{[2]})}.{a^{[1]}}^T})^T.dz^{[2]}*\color{blue}{\sigma ^{'}(z^{[2]})}.{a^{[0]}}^T$

From the above set of equations we can observe that as $i$ decreases from 5 to 1, the number of $\sigma^{'}$ terms increases from 1 to 5 in $dW^{[i]}$. Note that $\sigma^{'}$ is derivative of sigmoid function. It has maximum value of $0.25$. So, as $i$ decreases from 5 to 1, computation of $dW^{[i]}$ involves multiplication of increasing number of terms bounded by $0.25$. This will significantly bring down the magnitude of $dW^{[i]}$ as $i$ decreases from 5 to 1. This is a primary cause of vanishing gradients. This will hinder the update of $W$ at earlier layers thereby slowing the learning process. Sigmoid activation is the main culprit here. In fact, any activation that has similar behaviour to sigmoid will suffer from this problem. For eg, tanh also will exhibit this behaviour though tanh is slightly better as its outputs are centred around 0 unlike in sigmoid where outputs are centred around 0.5. Derivative of tanh has maximum value of 1.

What about ReLU?? Let's try it. Below is the plot of size of $dW$ for all neurons across all layers w.r.to solution of exercise1 where ReLU is used everywhere except at the output layer. At output layer, sigmoid is used. 
<img src='dw_relu_4layers.png'>

Similar plot for solution to exercise 2 with ReLU activation at all layers except at output layer where sigmoid is used is shown below. Also the speed of learning is depicted in this scenario.
<img src='dw_relu_5layers.png'>
<img src='speed_learning_relu.png'>

There seems to be an improvement with ReLU usage since many neurons from layers 1 and 2 are visible in the plots. But note that the magnitudes of size of $dW$ is again negligible, in fact across layers, thereby disrupting the learning process. Why this behaviour even when ReLU is used??

Remember the random initialization we have used for the layers: np.random.randn($n_{in}$, $n_{out}$)$*$0.01 where $n_{in}$ and $n_{out}$ are number of incoming and outgoing connections. Note that ReLU passes forward anything positive as it is and zeros out otherwise. Derivative of ReLU is constant $1$ for positive input and zero otherwise. We had argued in the introduction to this course that multiplication by 0.01 will prevent large weights and so avoid saturation of activation functions like sigmoid which in turn will produce non-zero gradients. But multiplication by 0.01 can make small values smaller and other values small relatively. This will cause inputs to ReLU to be small. Hence outputs from ReLU will be either small or zero. Since $dW^{[i]}$ depends on $a^{[i-1]}$ which are outputs from ReLU except at last layer, size of $dW^{[i]}$ will become small. This is possibly a reason for degradation in speed of learning even when ReLU is used.

So, apart from activation, initialization can also influence vanishing gradient problem. We will look at better initialization strategies now.

## Xavier/Glorot Initialization
To make sure that weights are appropriately randomly initialized, Xavier Glorot opined that the variance in the input and output should be equal. For instance, consider the linear neuron scenario:
$z$ = $\sum_{i=1}^{n_{in}}{W_ia_i}+b$ where z is the output of a single linear neuron, $W_i$'s are the weights from the $n_{in}$ incoming connections and $a_i$'s are the $n_{in}$ inputs. Let us assume $W_i$'s are identically independantly distributed and centred around zero. Similarly, $a_i$'s are independant and centred around zero. The assumption on $a_i$'s will be valid for symmetric activations like tanh. Note that for sigmoid, the outputs will be centred around 0.5 and so for the assumption to become valid, the outputs need to be translated by 0.5 to get centred around zero. Then
<br><br>
$var(z)$ = $var(\sum_{i=1}^{n_{in}}{W_ia_i}+b)$
<br>
&emsp;&emsp;&nbsp; &nbsp; = $\sum_{i=1}^{n_{in}}{var({W_ia_i})}$
<br>
&emsp;&emsp;&nbsp; &nbsp; = $\sum_{i=1}^{n_{in}}{E[W_i]^2var(a_i)+E[a_i]^2var(W_i)+var(W_i)var(a_i)}$ &emsp;&emsp;(see wikipedia article on variance)
<br>
&emsp;&emsp;&nbsp; &nbsp; = $\sum_{i=1}^{n_{in}}{var(W_i)var(a_i)}$
<br>
&emsp;&emsp;&nbsp; &nbsp; = $n_{in}var(W_i)var(a_i)$
<br><br>
i.e
<br>
$var(output)$ = $n_{in}var(W_i)var(output)$
<br>
This implies $var(W_i)$ = $\frac{1.0}{n_{in}}$

Similarly, if it is insisted that in the backward propagation the variance of incoming gradients at a layer should equal variance of outgoing gradients at that layer, we can derive
<br>
var($W_i$) = $\frac{1.0}{n_{out}}$ where $n_{out}$ is the number of outgoing connections (in the forward direction) from the layer under consideration.

Now both equations can be simultanously satisfied only when $n_{in}$ and $n_{out}$ are equal which may not always be the case. Hence, as a compromise, Xavier Glorot suggested to initialize weights randomly from a distribution with mean zero and variance $\frac{2.0}{n_{in}+n_{out}}$.

## He initialization for ReLU activation
ReLU is not a symmetric activation. Therefore the assumption that $a_i$'s are centred around zero is not valid for ReLU. 

Let $z^{[l]}$ be the random variable denoting linear output at a neuron in layer $l$. Let$W^{[l]}_i$'s be the weights at layer $l$ from the $n^{[l]}_{in}$ incoming connections into the neuron under consideration and $a^{[l-1]}_i$'s be the inputs to the neuron. These are ReLU outputs from the previous neuron. Let us assume $W^{[l]}_i$'s are identically independantly distributed and centred around zero. Let $a^{[l-1]}_i$'s be independant and identically distributed. Let $b^{[l]}$ be the bias. 

Then
<br>
$z^{[l]}$ = $\sum_{i=1}^{n^{[l]}_{in}}{W^{[l]}_ia^{[l-1]}_i}+b^{[l]}$ where $a^{[l-1]}_i$ = max(0, $z^{[l-1]}_i$)
<br>
$var(z^{[l]})$ = $n^{[l]}_{in}E[{(a^{[l-1]}})^2]var(W^{[l]})$&emsp;&emsp;(see wikipedia article on variance)

If we assume $W^{[l-1]}$ are identically independantly distributed and centred around zero and $b^{[l]}$ = $0$, then $z^{[l-1]}$ is distributed symmetrically around zero. Hence $E[{(a^{[l-1]}})^2]$ = $\frac{1}{2}var(z^{[l-1]})$. This implies
<br><br>
$var(z^{[l]})$ = $n^{[l]}_{in}\frac{1}{2}var(z^{[l-1]})var(W^{[l]})$
<br><br>
Therefore,
<br><br>
$var(z^{[L]})$ = $var(z^{[1]})(\prod_{l=2}^{L}{\frac{1}{2}n^{[l]}_{in}var(W^{[l]})})$ 

For equating $var(z^{[L]})$ to $var(z^{[1]})$ we require $\frac{1}{2}n^{[l]}_{in}var(W^{[l]})$ = 1 which implies $var(W^{[l]})$ = $\frac{1.0}{2n^{[l]}_{in}}$.

For the first layer (l = 1), we should have $n^{[l]}_{in}var(W^{[l]})$ = $1$ because there is no ReLU applied on the input signal. But the factor $\frac{1}{2}$ does not matter if it just exists on one layer.

Further, He et al. argues that either forward propagation analysis done above is sufficent or backward propagation analysis that can be done similarly is sufficent. Both are simultaneously not required. Therefore for ReLU activation, weights at layer $l$ can be intialized from a distribution with zero mean and variance equal to $\frac{1.0}{2n^{[l]}_{in}}$. Bias should be initialized to zero.

## Exercise
1. Consider a model similar to exercise 2 but with ReLU activations in layers 1 to 4 and sigmoid activation at layer 5. Use He initialization and implement the model.

In [174]:
class Linear(Layers): # inherits from abstract class Layers
    def __init__(self, in_features, out_features, bias = True, regularization = None): # constructor
        super().__init__()
        self.in_features = in_features # initialize number of incoming features
        self.out_features = out_features # initialize number of outgoing features
        sigma = np.sqrt(1./(2*in_features))
        self.weight = sigma * np.random.randn(out_features, in_features) # initialize weight matrix
        if bias:
            self.bias = np.zeros((out_features, 1)) # initialize bias if required
        else:
            self.bias = None # if bias not required, set it to None
        self.regularization = regularization # initialize regularization
        if self.regularization is not None: 
            self.reg_penalty = 0 # if valid regularization at this layer, set the regularization penalty
                                 # at this layer initially to zero
            
        self.dw = np.empty_like(self.weight) # intialize dw to empty
        self.db = np.empty_like(self.bias) if self.bias is not None else None 
                                            # initialize db to empty if bias == True
        
    def forward(self, x): # forward method overridden; x is the incoming activation. 
                          # shape of x is (num of activations, num of samples)
            
        m = x.shape[1]    # number of training examples
        self.x = x # x is required for backward. We don't need a separate cache. We can store it in the object.
        
        output = self.weight @ x # computation of the linear part Wx                                 
        if self.bias is not None:            
            output += self.bias # add to Wx bias if bias == True
                                # Note that we don't apply non-linearity here as this layer computes only Wx+b
        
        #update regularization penalty at this layer
        if self.regularization is None:
            pass
        elif self.regularization == 'L2':
            self.reg_penalty = args.lamda/(2*m) * np.sum(self.weight**2)
        elif self.regularization == 'L1':
            self.reg_penalty = args.lamda/m * np.sum(np.abs(self.weight))
        else:
            raise ValueError(f'Regularization method{self.regularization} not defined')
            
        return output # return forward output as the next layer in the model will require it     
    
    # Backward of this layer will receive dz. Note that at this layer z = Wx+b. So backward will compute
    # dw, db and dx. To compute dW, x is required. That's why x was stored in forward. To compute dx, W is 
    # required. This is already available in self.weight
    
    def backward(self, dz): # backward method overridden
        m = dz.shape[1]     # number of training examples
        self.dw = dz @ self.x.T # compute dw
        
        # add derivative of regularization penalty at this layer w.r.to w
        if self.regularization is None:
            pass
        elif self.regularization == 'L1':
            signw = np.sign(self.weight)
            signw[signw == 0] = 1
            self.dw += args.lamda/m * signw
        elif self.regularization == 'L2':
            self.dw += args.lamda/m * self.weight
        else:
            raise ValueError(f'Regularization method{self.regularization} not defined')
            
        if self.bias is not None: # compute db if bias == True
            self.db = np.sum(dz, axis = 1, keepdims = True) 
            
        dx = self.weight.T @ dz  # compute dx     
        return dx # we only return dx as this this required for chain rule in the next layer 
                  # in backward direction. dw and db are kept available in this object which will directly
                  # be used by update_params for updating weights and biases
    
    # update parameters in this layer 
    def update_params(self, learning_rate = 0.005):
        self.weight -= learning_rate*self.dw
        if self.bias is not None:
            self.bias -= learning_rate*self.db

In [175]:
class Model(Layers):
    def __init__(self, in_features):
        super().__init__()
        # self.fc1 = Linear(in_features, 32, regularization = 'L2')
        self.fc1 = Linear(in_features, 30, regularization='L2')
        self.ReLU1 = NonLinear('ReLU')
        self.fc2 = Linear(30, 30)
        self.ReLU2 = NonLinear('ReLU')
        self.fc3 = Linear(30, 30)
        self.ReLU3 = NonLinear('ReLU')
        self.fc4 = Linear(30, 30)
        self.ReLU4 = NonLinear('ReLU')
        self.fc5 = Linear(30, 1)
        self.Sigmoid = NonLinear('Sigmoid')
        
    def forward(self, x, train=True):
        x = self.fc1(x) # Note that we made classes callable which automatically calls forward method
                        # That's why we could call fc1(x) instead of fc1.forward(x). Calls below 
                        # are in similar line.
                        # we could call fc1.forward(x) also.
        x = self.ReLU1(x)
        x = self.fc2(x)
        x = self.ReLU2(x)
        x = self.fc3(x)
        x = self.ReLU3(x)
        x = self.fc4(x)
        x = self.ReLU4(x)
        x = self.fc5(x)
        x = self.Sigmoid(x)
        return x
    
    def loss(self, output, y):
#         print("Model Loss")
        m = output.shape[1]
        L = -(1./m) * np.sum(y*np.log(output) + (1-y)*np.log(1-output)) # compute loss
        for att in self.__dict__:
            if hasattr(att, 'reg_penalty'):
                L += att.reg_penalty
        return L
    
    def backward(self, output, y):
        epsilon = 1e-6
        m = output.shape[1]
        d_output = (1./m) * (output-y) * (1./((output*(1-output))+epsilon)) # compute da        
        dz = self.Sigmoid.backward(d_output)
        dx = self.fc5.backward(dz)
        dz = self.ReLU4.backward(dx)
        dx = self.fc4.backward(dz)
        dz = self.ReLU3.backward(dx)
        dx = self.fc3.backward(dz)
        dz = self.ReLU2.backward(dx)
        dx = self.fc2.backward(dz)
        dz = self.ReLU1.backward(dx)
        dx = self.fc1.backward(dz)
    
    def update_params(self, learning_rate = 0.005):
        self.fc1.update_params(learning_rate)
        self.fc2.update_params(learning_rate)
        self.fc3.update_params(learning_rate)
        self.fc4.update_params(learning_rate)
        self.fc5.update_params(learning_rate)
        
    
    def __call__(self, *x, train=True):
        return self.forward(*x, train=train)

In [176]:
# instantiate the ArgumentParser object
parser = argparse.ArgumentParser(description='Train a fully connected network with regularization')
# add arguments
parser.add_argument('--miter', metavar='N', type=int, default=200, help='max number of iterations to train')
parser.add_argument('--alpha', metavar='LEARNING_RATE', type=float, default=0.001, help='initial learning rate')
parser.add_argument('--lamda', metavar='LAMBDA', type=float, default=1., help='regularization parameter')
parser.add_argument('--print_freq', metavar='N', type=int, default=300, help='print model loss every print_freq iterations')

# parse the arguments. 
# Since we cannot invoke the code written in jupyter directly from command-line, 
# we can specify the required arguments in the call to parse_args as shown below with other arguments 
# left out to use their default values.
args = parser.parse_args('--miter 2200 --alpha .008 --lamda 1.5'.split()) # you may play with this code by changing
                                                                        # the arguments as required

In [177]:
def main(): # main function to train and test the model    
    
    global args
    # load train data
    x, y = load_train_data()
    x = flatten(x)
    x = x/255. # normalize the data to [0, 1]     
    
    # Instantiate the model
    my_model = Model(x.shape[0])
    
    # train the model
    train(my_model, x, y)
    
    # test the model
    print(f'train accuracy: {test_model(my_model, x, y) * 100:.2f}%')

    x, y = load_test_data()
    x = flatten(x)
    x = x/255. # normalize the data to [0, 1]
    print(f'test accuracy: {test_model(my_model, x, y) * 100:.2f}%')
    
    return
    
if __name__ == '__main__':
    main()

Loss at iteration 0:	0.6930
Loss at iteration 300:	0.6274
Loss at iteration 600:	0.5294
Loss at iteration 900:	0.4088
Loss at iteration 1200:	0.2337
Loss at iteration 1500:	0.0490
Loss at iteration 1800:	0.0130
Loss at iteration 2100:	0.0061
train accuracy: 100.00%
test accuracy: 78.00%


2. Consider exercise 2. Use Xavier initialization and implement the model. Make sure the outputs at each layer are translated to be centred around zero.

In [231]:
class Linear(Layers): # inherits from abstract class Layers
    def __init__(self, in_features, out_features, bias = True, regularization = None): # constructor
        super().__init__()
        self.in_features = in_features # initialize number of incoming features
        self.out_features = out_features # initialize number of outgoing features
        sigma = np.sqrt(2./(in_features + out_features))
        self.weight = sigma * np.random.randn(out_features, in_features) # initialize weight matrix
        if bias:
            self.bias = np.zeros((out_features, 1)) # initialize bias if required
        else:
            self.bias = None # if bias not required, set it to None
        self.regularization = regularization # initialize regularization
        if self.regularization is not None: 
            self.reg_penalty = 0 # if valid regularization at this layer, set the regularization penalty
                                 # at this layer initially to zero
            
        self.dw = np.empty_like(self.weight) # intialize dw to empty
        self.db = np.empty_like(self.bias) if self.bias is not None else None 
                                            # initialize db to empty if bias == True
        
    def forward(self, x): # forward method overridden; x is the incoming activation. 
                          # shape of x is (num of activations, num of samples)
            
        m = x.shape[1]    # number of training examples
        self.x = x # x is required for backward. We don't need a separate cache. We can store it in the object.
        
        output = self.weight @ x # computation of the linear part Wx                                 
        if self.bias is not None:            
            output += self.bias # add to Wx bias if bias == True
                                # Note that we don't apply non-linearity here as this layer computes only Wx+b
        
        #update regularization penalty at this layer
        if self.regularization is None:
            pass
        elif self.regularization == 'L2':
            self.reg_penalty = args.lamda/(2*m) * np.sum(self.weight**2)
        elif self.regularization == 'L1':
            self.reg_penalty = args.lamda/m * np.sum(np.abs(self.weight))
        else:
            raise ValueError(f'Regularization method{self.regularization} not defined')
            
        return output # return forward output as the next layer in the model will require it     
    
    # Backward of this layer will receive dz. Note that at this layer z = Wx+b. So backward will compute
    # dw, db and dx. To compute dW, x is required. That's why x was stored in forward. To compute dx, W is 
    # required. This is already available in self.weight
    
    def backward(self, dz): # backward method overridden
        m = dz.shape[1]     # number of training examples
        self.dw = dz @ self.x.T # compute dw
        
        # add derivative of regularization penalty at this layer w.r.to w
        if self.regularization is None:
            pass
        elif self.regularization == 'L1':
            signw = np.sign(self.weight)
            signw[signw == 0] = 1
            self.dw += args.lamda/m * signw
        elif self.regularization == 'L2':
            self.dw += args.lamda/m * self.weight
        else:
            raise ValueError(f'Regularization method{self.regularization} not defined')
            
        if self.bias is not None: # compute db if bias == True
            self.db = np.sum(dz, axis = 1, keepdims = True) 
            
        dx = self.weight.T @ dz  # compute dx     
        return dx # we only return dx as this this required for chain rule in the next layer 
                  # in backward direction. dw and db are kept available in this object which will directly
                  # be used by update_params for updating weights and biases
    
    # update parameters in this layer 
    def update_params(self, learning_rate = 0.005):
        self.weight -= learning_rate*self.dw
        if self.bias is not None:
            self.bias -= learning_rate*self.db

In [232]:
class Model(Layers):
    def __init__(self, in_features):
        super().__init__()
        self.fc1 = Linear(in_features, 30, regularization = 'L2')
#         self.fc1 = Linear(in_features, 30)
        self.sigmoid1 = NonLinear('Sigmoid')
        self.fc2 = Linear(30, 30)
        self.sigmoid2 = NonLinear('Sigmoid')
        self.fc3 = Linear(30, 30)
        self.sigmoid3 = NonLinear('Sigmoid')
        self.fc4 = Linear(30, 30)
        self.sigmoid4 = NonLinear('Sigmoid')
        self.fc5 = Linear(30, 1)
        self.sigmoid5 = NonLinear('Sigmoid')
        
    def forward(self, x, train=True):
        x = self.fc1(x) # Note that we made classes callable which automatically calls forward method
                        # That's why we could call fc1(x) instead of fc1.forward(x). Calls below 
                        # are in similar line.
                        # we could call fc1.forward(x) also.
        x = self.sigmoid1(x) - .5
        x = self.fc2(x)
        x = self.sigmoid2(x) - .5
        x = self.fc3(x)
        x = self.sigmoid3(x) - .5
        x = self.fc4(x)
        x = self.sigmoid4(x) - .5
        x = self.fc5(x)
        x = self.sigmoid5(x)
        return x
    
    def loss(self, output, y):
        m = output.shape[1]
        L = -(1./m) * np.sum(y*np.log(output) + (1-y)*np.log(1-output)) # compute loss
        for att in self.__dict__:
            if hasattr(att, 'reg_penalty'):
                L += att.reg_penalty
        return L
    
    def backward(self, output, y):
        epsilon = 1e-6
        m = output.shape[1]
        d_output = (1./m) * (output-y) * (1./((output*(1-output))+epsilon)) # compute da        
        dz = self.sigmoid5.backward(d_output)
        dx = self.fc5.backward(dz)
        dz = self.sigmoid4.backward(dx)
        dx = self.fc4.backward(dz)
        dz = self.sigmoid3.backward(dx)
        dx = self.fc3.backward(dz)
        dz = self.sigmoid2.backward(dx)
        dx = self.fc2.backward(dz)
        dz = self.sigmoid1.backward(dx)
        dx = self.fc1.backward(dz)
    
    def update_params(self, learning_rate = 0.005):
        self.fc1.update_params(learning_rate)
        self.fc2.update_params(learning_rate)
        self.fc3.update_params(learning_rate)
        self.fc4.update_params(learning_rate)
        self.fc5.update_params(learning_rate)
        
    
    def __call__(self, *x, train=True):
        return self.forward(*x, train=train)

In [233]:
# instantiate the ArgumentParser object
parser = argparse.ArgumentParser(description='Train a fully connected network with regularization')
# add arguments
parser.add_argument('--miter', metavar='N', type=int, default=200, help='max number of iterations to train')
parser.add_argument('--alpha', metavar='LEARNING_RATE', type=float, default=0.001, help='initial learning rate')
parser.add_argument('--lamda', metavar='LAMBDA', type=float, default=1., help='regularization parameter')
parser.add_argument('--print_freq', metavar='N', type=int, default=300, help='print model loss every print_freq iterations')

# parse the arguments. 
# Since we cannot invoke the code written in jupyter directly from command-line, 
# we can specify the required arguments in the call to parse_args as shown below with other arguments 
# left out to use their default values.
args = parser.parse_args('--miter 23000 --alpha .01'.split()) # you may play with this code by changing
                                                                        # the arguments as required

def main(): # main function to train and test the model    
    
    global args
    # load train data
    x, y = load_train_data()
    x = flatten(x)
    x = x/255. # normalize the data to [0, 1]     
    
    # Instantiate the model
    my_model = Model(x.shape[0])
    
    # train the model
    train(my_model, x, y)
    
    # test the model
    print(f'train accuracy: {test_model(my_model, x, y) * 100:.2f}%')

    x, y = load_test_data()
    x = flatten(x)
    x = x/255. # normalize the data to [0, 1]
    print(f'test accuracy: {test_model(my_model, x, y) * 100:.2f}%')
    
    return
    
if __name__ == '__main__':
    main()

Loss at iteration 0:	0.6930
Loss at iteration 300:	0.6531
Loss at iteration 600:	0.6458
Loss at iteration 900:	0.6444
Loss at iteration 1200:	0.6440
Loss at iteration 1500:	0.6439
Loss at iteration 1800:	0.6438
Loss at iteration 2100:	0.6437
Loss at iteration 2400:	0.6436
Loss at iteration 2700:	0.6435
Loss at iteration 3000:	0.6434
Loss at iteration 3300:	0.6433
Loss at iteration 3600:	0.6432
Loss at iteration 3900:	0.6430
Loss at iteration 4200:	0.6429
Loss at iteration 4500:	0.6428
Loss at iteration 4800:	0.6427
Loss at iteration 5100:	0.6425
Loss at iteration 5400:	0.6424
Loss at iteration 5700:	0.6423
Loss at iteration 6000:	0.6421
Loss at iteration 6300:	0.6419
Loss at iteration 6600:	0.6418
Loss at iteration 6900:	0.6416
Loss at iteration 7200:	0.6414
Loss at iteration 7500:	0.6412
Loss at iteration 7800:	0.6410
Loss at iteration 8100:	0.6407
Loss at iteration 8400:	0.6404
Loss at iteration 8700:	0.6401
Loss at iteration 9000:	0.6398
Loss at iteration 9300:	0.6395
Loss at iterat

#### All solutions/answers/analysis to be done in this notebook only